In [ ]:
import os
import torch
from torch import nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm

%run Model.ipynb
%run DataLoader.ipynb

cudnn.benchmark = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
images_dir = "/home/mj/HardDisk/Github/Image_Compressor/Dataset/Train/train"
outputs_dir = "/home/mj/HardDisk/Github/Image_Compressor/Model"
batch_size = 8
threads = 4
lr = 5e-4
num_epochs = 15
patch_size = 128
jpeg_quality = 10
use_augmentation = True
use_fast_loader = False
### 欠初始一些參數


model = My_Model()

model = model.to(device)
criterion = nn.MSELoss(reduction='sum')

optimizer = optim.Adam([
    {'params': model.base.parameters()},
    {'params': model.last.parameters(), 'lr': lr * 0.1},
], lr=lr)

dataset = Dataset(images_dir, patch_size, jpeg_quality, use_augmentation, use_fast_loader)
dataloader = DataLoader(dataset=dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=threads,
                        pin_memory=True,
                        drop_last=True)


for epoch in range(num_epochs):
    epoch_losses = AverageMeter()
    target_losses = AverageMeter()
    
    with tqdm(total=(len(dataset) - len(dataset) % batch_size)) as _tqdm:
        _tqdm.set_description('epoch: {}/{}'.format(epoch + 1, num_epochs+6))
        for data in dataloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            preds = model(inputs)

            loss = criterion(preds, labels)
            target = criterion(inputs, labels)
            
            epoch_losses.update(loss.item(), len(inputs))
            target_losses.update(target.item(), len(inputs))


            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _tqdm.set_postfix(loss='{:.6f}, target{:.6f}'.format(epoch_losses.avg, target_losses.avg))
            _tqdm.update(len(inputs))

    torch.save(model.state_dict(), os.path.join(outputs_dir, 'FastARCNN_epoch_{}.pth'.format(epoch)))
    

optimizer = optim.Adam([
    {'params': model.base.parameters()},
    {'params': model.last.parameters(), 'lr': lr * 0.5},
], lr=lr*0.5)

for epoch in range(num_epochs, num_epochs+3):
    epoch_losses = AverageMeter()
    target_losses = AverageMeter()
    
    with tqdm(total=(len(dataset) - len(dataset) % batch_size)) as _tqdm:
        _tqdm.set_description('epoch: {}/{}'.format(epoch + 1, num_epochs+6))
        for data in dataloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            preds = model(inputs)

            loss = criterion(preds, labels)
            target = criterion(inputs, labels)
            
            epoch_losses.update(loss.item(), len(inputs))
            target_losses.update(target.item(), len(inputs))


            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _tqdm.set_postfix(loss='{:.6f}, target{:.6f}'.format(epoch_losses.avg, target_losses.avg))
            _tqdm.update(len(inputs))

    torch.save(model.state_dict(), os.path.join(outputs_dir, 'FastARCNN_epoch_{}.pth'.format(epoch)))


optimizer = optim.Adam([
    {'params': model.base.parameters()},
    {'params': model.last.parameters(), 'lr': lr * 0.1},
], lr=lr*0.1)
for epoch in range(num_epochs+3, num_epochs+6):
    epoch_losses = AverageMeter()
    target_losses = AverageMeter()
    
    with tqdm(total=(len(dataset) - len(dataset) % batch_size)) as _tqdm:
        _tqdm.set_description('epoch: {}/{}'.format(epoch + 1, num_epochs+6))
        for data in dataloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            preds = model(inputs)

            loss = criterion(preds, labels)
            target = criterion(inputs, labels)
            
            epoch_losses.update(loss.item(), len(inputs))
            target_losses.update(target.item(), len(inputs))


            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _tqdm.set_postfix(loss='{:.6f}, target{:.6f}'.format(epoch_losses.avg, target_losses.avg))
            _tqdm.update(len(inputs))

    torch.save(model.state_dict(), os.path.join(outputs_dir, 'FastARCNN_epoch_{}.pth'.format(epoch)))

In [ ]:
import matplotlib.pyplot as plt
import torchvision

In [ ]:
patch_size = 128

test_dir = "/home/mj/HardDisk/Github/Image_Compressor/Dataset/DIV2K_valid_HR"
test_dataset = Dataset(images_dir, patch_size, jpeg_quality, False, use_fast_loader)
test_dataloader = DataLoader(dataset=test_dataset,
                        batch_size=1,
                        shuffle=True,
                        num_workers=1,
                        pin_memory=True,
                        drop_last=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for data in test_dataloader:
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    preds = model(inputs)
    
#     print((inputs*255).type(torch.cuda.ByteTensor))
#     print((labels*255).type(torch.cuda.ByteTensor))
#     print((preds*255).type(torch.cuda.ByteTensor))

# torchvision.utils.make_grid(images, nrow=5).permute(1, 2, 0)

    inputs = (inputs*255).cpu().type(torch.ByteTensor)
    labels = (labels*255).cpu().type(torch.ByteTensor)
    preds = (preds*255).cpu().type(torch.ByteTensor)
    
    plt.imshow(torchvision.utils.make_grid(inputs,nrow=1).permute(1, 2, 0))
    plt.title("JPG")
    plt.show()
    plt.imshow(torchvision.utils.make_grid(labels,nrow=1).permute(1, 2, 0))
    plt.title("Ori")
    plt.show()
    plt.imshow(torchvision.utils.make_grid(preds,nrow=1).permute(1, 2, 0))
    plt.title("Fixed")
    plt.show()
    break